In [1]:
import os
import numpy as np
import pandas as pd
import json
import joblib
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy import interpolate

In [2]:
# 1. 数据加载函数
def load_all_data(base_path):
    """
    加载所有CSV文件并处理为训练格式
    """
    all_samples = []
    all_labels = []
    
    for action_id in range(6):  # 0-5.csv
        file_path = os.path.join(base_path, f"{action_id}.csv")
        try:
            df = pd.read_csv(file_path)
            
            # 验证数据格式
            required_columns = ['acc_x', 'acc_y', 'acc_z', 'gyr_x', 'gyr_y', 'gyr_z', 'sample', 'label']
            if not all(col in df.columns for col in required_columns):
                print(f"文件 {file_path} 缺少必要的列，跳过")
                continue
            
            # 按样本分组
            for sample_id in df['sample'].unique():
                sample_data = df[df['sample'] == sample_id]
                
                # 确保每个样本有100个数据点
                if len(sample_data) < 100:
                    # 插值补全到100点
                    new_index = np.linspace(0, 1, 100)
                    old_index = np.linspace(0, 1, len(sample_data))
                    
                    interpolated = pd.DataFrame()
                    for col in ['acc_x', 'acc_y', 'acc_z', 'gyr_x', 'gyr_y', 'gyr_z']:
                        f = interpolate.interp1d(old_index, sample_data[col], kind='cubic')
                        interpolated[col] = f(new_index)
                    
                    sample_data = interpolated
                elif len(sample_data) > 100:
                    sample_data = sample_data.head(100)
                
                # 提取传感器数据
                sensor_data = sample_data[['acc_x', 'acc_y', 'acc_z', 'gyr_x', 'gyr_y', 'gyr_z']].values
                all_samples.append(sensor_data)
                all_labels.append(action_id)
                
        except Exception as e:
            print(f"加载文件 {file_path} 时出错: {e}")
    
    return np.array(all_samples), np.array(all_labels)

In [3]:
# 2. 数据预处理
def preprocess_data(X, y):
    """
    数据预处理：标准化和分割
    """
    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )
    
    # 标准化数据
    train_2d = X_train.reshape(-1, X_train.shape[-1])
    scaler = StandardScaler()
    scaler.fit(train_2d)
    
    # 应用相同的标准化到所有数据
    def scale_dataset(dataset):
        original_shape = dataset.shape
        flat = dataset.reshape(-1, dataset.shape[-1])
        scaled = scaler.transform(flat)
        return scaled.reshape(original_shape)
    
    X_train_scaled = scale_dataset(X_train)
    X_test_scaled = scale_dataset(X_test)
    
    # One-hot编码标签
    num_classes = len(np.unique(y))  # 获取实际类别数
    y_train_cat = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)
    y_test_cat = tf.keras.utils.to_categorical(y_test, num_classes=num_classes)
    
    return X_train_scaled, X_test_scaled, y_train_cat, y_test_cat, scaler, num_classes

In [4]:
# 3. 模型构建
def create_model(input_shape, num_classes):
    """
    创建CNN-LSTM模型
    """
    model = models.Sequential([
        layers.Input(shape=input_shape),
        
        # 1D CNN 特征提取
        layers.Conv1D(32, kernel_size=5, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(0.3),
        
        layers.Conv1D(64, kernel_size=3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(0.4),
        
        # LSTM 时序建模
        layers.LSTM(64, return_sequences=True),
        layers.Dropout(0.5),
        layers.LSTM(32),
        layers.Dropout(0.5),
        
        # 分类层
        layers.Dense(32, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.summary()
    return model

In [5]:
# 4. 保存训练结果
def save_training_results(model, scaler, history, test_acc, label_names, num_classes):
    """
    保存模型、标准化器和训练元数据
    """
    # 创建保存目录
    save_dir = "saved_model"
    os.makedirs(save_dir, exist_ok=True)
    
    # 保存模型
    model.save(os.path.join(save_dir, "gesture_model.h5"))
    
    # 保存标准化器
    joblib.dump(scaler, os.path.join(save_dir, "scaler.pkl"))
    
    # 保存标签映射
    label_mapping = {i: name for i, name in enumerate(label_names)}
    with open(os.path.join(save_dir, "label_mapping.json"), 'w') as f:
        json.dump(label_mapping, f, indent=2)
    
    # 保存训练历史
    history_df = pd.DataFrame(history.history)
    history_df.to_csv(os.path.join(save_dir, "training_history.csv"), index=False)
    
    # 保存元数据
    metadata = {
        "test_accuracy": float(test_acc),
        "input_shape": model.input_shape[1:],
        "num_classes": num_classes,
        "created_at": pd.Timestamp.now().isoformat()
    }
    with open(os.path.join(save_dir, "metadata.json"), 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"所有训练结果已保存到 {save_dir} 目录")

In [8]:
# 主训练函数
def train_gesture_model(data_path, label_names):
    """
    完整训练流程
    """
    print("=== 开始训练手势识别模型 ===")
    
    # 1. 加载数据
    print("加载数据...")
    X, y = load_all_data(data_path)
    print(f"加载完成: {len(X)} 个样本")
    
    # 2. 预处理
    print("预处理数据...")
    X_train, X_test, y_train, y_test, scaler, num_classes = preprocess_data(X, y)
    print(f"训练集: {X_train.shape[0]} 样本, 测试集: {X_test.shape[0]} 样本, 类别数: {num_classes}")
    
    # 3. 构建模型
    print("构建模型...")
    model = create_model((100, 6), num_classes)
    
    # 4. 训练模型
    print("训练模型...")
    early_stopping = callbacks.EarlyStopping(
        monitor='val_loss', patience=20, restore_best_weights=True
    )
    reduce_lr = callbacks.ReduceLROnPlateau(
        monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6
    )
    model_checkpoint = callbacks.ModelCheckpoint(
        'best_model.h5', save_best_only=True, monitor='val_accuracy'
    )
    
    history = model.fit(
        X_train, y_train,
        epochs=200,
        batch_size=16,
        validation_data=(X_test, y_test),
        callbacks=[early_stopping, reduce_lr, model_checkpoint],
        verbose=1
    )
    
    # 5. 评估模型
    print("评估模型...")
    model.load_weights('best_model.h5')  # 加载最佳模型
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    print(f"测试准确率: {test_acc:.4f}")
    
    # 6. 保存结果
    print("保存训练结果...")
    save_training_results(model, scaler, history, test_acc, label_names, num_classes)
    
    print("=== 训练完成 ===")

In [9]:
if __name__ == "__main__":
    # 配置参数
    DATA_PATH = "sample"  # 包含0.csv到5.csv的目录
    LABEL_NAMES = ["up", "down", "left", "right", "circle_clock", "circle_anticlock"]
    
    # 开始训练
    train_gesture_model(DATA_PATH, LABEL_NAMES)

=== 开始训练手势识别模型 ===
加载数据...
加载完成: 60 个样本
预处理数据...
训练集: 48 样本, 测试集: 12 样本, 类别数: 6
构建模型...


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 100, 32)        │           992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 100, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 50, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 50, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 50, 64)         │         6,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 50, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 25, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 25, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 25, 64)         │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 25, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 6)              │           198 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,278 (212.02 KB)

 Trainable params: 54,086 (211.27 KB)

...

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.4271 - loss: 1.6548 - val_accuracy: 0.7500 - val_loss: 1.7351 - learning_rate: 0.0010
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.3594 - loss: 1.6495 - val_accuracy: 0.7500 - val_loss: 1.7138 - learning_rate: 0.0010
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.4427 - loss: 1.6084 - val_accuracy: 0.7500 - val_loss: 1.6864 - learning_rate: 0.0010
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.6042 - loss: 1.5051 - val_accuracy: 0.6667 - val_loss: 1.6520 - learning_rate: 0.0010
Epoch 7/200
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.6250 - loss: 1.4797

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.5859 - loss: 1.4895 - val_accuracy: 0.8333 - val_loss: 1.6084 - learning_rate: 0.0010
Epoch 8/200
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.7500 - loss: 1.4322

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - accuracy: 0.6458 - loss: 1.4337 - val_accuracy: 0.9167 - val_loss: 1.5529 - learning_rate: 0.0010
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5755 - loss: 1.3943 - val_accuracy: 0.9167 - val_loss: 1.4899 - learning_rate: 0.0010
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.6953 - loss: 1.2856 - val_accuracy: 0.9167 - val_loss: 1.4170 - learning_rate: 0.0010
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.7734 - loss: 1.2153 - val_accuracy: 0.9167 - val_loss: 1.3313 - learning_rate: 0.0010
Epoch 12/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.7266 - loss: 1.1733 - val_accuracy: 0.9167 - val_loss: 1.2469 - learning_rate: 0.0010
Epoch 13/200
1/3 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.8750 - loss: 1.1593

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.8750 - loss: 1.0847 - val_accuracy: 1.0000 - val_loss: 1.1688 - learning_rate: 0.0010
Epoch 14/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.7708 - loss: 1.0144 - val_accuracy: 1.0000 - val_loss: 1.0911 - learning_rate: 0.0010
Epoch 15/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.8880 - loss: 0.8999 - val_accuracy: 1.0000 - val_loss: 1.0115 - learning_rate: 0.0010
Epoch 16/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 1.0000 - loss: 0.7772 - val_accuracy: 1.0000 - val_loss: 0.9250 - learning_rate: 0.0010
Epoch 17/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.9349 - loss: 0.7021 - val_accuracy: 1.0000 - val_loss: 0.8333 - learning_rate: 0.0010
Epoch 18/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.9323 - loss: 0.6378 - val_accuracy: 1.0000 - val_loss: 0.7515 - learning_rate: 0.0010
Epoch 19/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.9635 - loss: 0.5938 - val_accuracy: 1.0000

所有训练结果已保存到 saved_model 目录
=== 训练完成 ===
